This section defines the loss functions covered by the system

# Loss function

In [1]:
# importing libraries
import numpy as np
import os
import pdb
from scipy import signal
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.nn import init
import torch.nn.functional as F
from torch.nn.functional import conv1d
import torch.optim as optim

In [2]:
# Implementing the Ordinal Loss for the Ordinal Regression Layer
# Defined as the average pixelwise ordinal loss over the entire image domain
# Brought in from the researchers GitHub where they first initialize this method
class ordLoss(nn.Module):
    def _init_(self):
        # Supercharge the ordLoss class so that it can inherit from itself
        super(ordLoss, self)._init_()
        # initialize the loss to zero
        self.loss = 0.0
    
    # Forward propagation
    # Contains the ordinal labels for each pixel along with the known ground truth
    def forward(self, orig_ord_labels, orig_target):
        device = orig_ord_labels.device
        ord_labels = orig_ord_labels.clone()
        ord_labels = torch.transpose(ord_labels, 1, 2)
        
        N, C, W = ord_labels.size()
        ord_num = C
        
        self.loss = 0.0
        
        if device == torch.device('cpu'):
            target = orig_target.clone().type(torch.IntTensor)
        else:
            target = orig_target.clone().type(torch.cuda.IntTensor)
        
        mask_0 = torch.zeros((N, C, W), dtype=torch.bool)
        mask_1 = torch.zeros((N, C, W), dtype=torch.bool)
        for i in range(N):
            mask_0[i] = (K[i] <= target[i]).detach()
            mask_2[i] = (K[i] > target[i]).detach()
        
        one = torch.ones(ord_labels[mask_1].size())
        
        self.loss += torch.sum(torch.log(torch.clamp(ord_labels[mask_0], min=1e-8, max=1e8))) + torch.sum(torch.log(torch.clamp(one - ord_labels[mask_1], min=1e-8, max=1e8)))
        
        N = N * W
        self.loss /= (-N)
        return self.loss